<a href="https://colab.research.google.com/github/microsoft/autogen/blob/main/notebook/agentchat_human_feedback.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Auto Generated Agent Chat: Task Solving with Code Generation, Execution, Debugging & Human Feedback

AutoGen offers conversable agents powered by LLM, tool, or human, which can be used to perform tasks collectively via automated chat. This framework allows tool use and human participation through multi-agent conversation.
Please find documentation about this feature [here](https://microsoft.github.io/autogen/docs/Use-Cases/agent_chat).

In this notebook, we demonstrate how to use `AssistantAgent` and `UserProxyAgent` to solve a challenging math problem with human feedback. Here `AssistantAgent` is an LLM-based agent that can write Python code (in a Python coding block) for a user to execute for a given task. `UserProxyAgent` is an agent which serves as a proxy for a user to execute the code written by `AssistantAgent`. By setting `human_input_mode` properly, the `UserProxyAgent` can also prompt the user for feedback to `AssistantAgent`. For example, when `human_input_mode` is set to "ALWAYS", the `UserProxyAgent` will always prompt the user for feedback. When user feedback is provided, the `UserProxyAgent` will directly pass the feedback to `AssistantAgent`. When no user feedback is provided, the `UserProxyAgent` will execute the code written by `AssistantAgent` and return the execution results (success or failure and corresponding outputs) to `AssistantAgent`.

## Requirements

AutoGen requires `Python>=3.8`. To run this notebook example, please install:
```bash
pip install pyautogen
```

In [3]:
%%capture --no-stderr
%pip install pyautogen~=0.2.0b4

## Set your API Endpoint

The [`config_list_from_json`](https://microsoft.github.io/autogen/docs/reference/oai/openai_utils#config_list_from_json) function loads a list of configurations from an environment variable or a json file.

In [4]:
import autogen

config_list = [
    {
        'base_url': "https://narrative-panels-terrain-nike.trycloudflare.com/openai",
        'api_key': "NULL"
    }
]

It first looks for environment variable "OAI_CONFIG_LIST" which needs to be a valid json string. If that variable is not found, it then looks for a json file named "OAI_CONFIG_LIST". It filters the configs by models (you can filter by other keys as well). Only the models with matching names are kept in the list based on the filter condition.

The config list looks like the following:
```python
config_list = [
    {
        'model': 'gpt-4',
        'api_key': '<your OpenAI API key here>',
    },
    {
        'model': 'gpt-3.5-turbo',
        'api_key': '<your Azure OpenAI API key here>',
        'base_url': '<your Azure OpenAI API base here>',
        'api_type': 'azure',
        'api_version': '2023-08-01-preview',
    },
    {
        'model': 'gpt-3.5-turbo-16k',
        'api_key': '<your Azure OpenAI API key here>',
        'base_url': '<your Azure OpenAI API base here>',
        'api_type': 'azure',
        'api_version': '2023-08-01-preview',
    },
]
```

You can set the value of config_list in any way you prefer. Please refer to this [notebook](https://github.com/microsoft/autogen/blob/main/notebook/oai_openai_utils.ipynb) for full code examples of the different methods.

## Construct Agents

We construct the assistant agent and the user proxy agent.

In [5]:
# create an AssistantAgent instance named "assistant"
assistant = autogen.AssistantAgent(
    name="assistant",
    llm_config={
        "cache_seed": 41,
        "config_list": config_list,
    }
)
# create a UserProxyAgent instance named "user_proxy"
user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    human_input_mode="ALWAYS",
    is_termination_msg=lambda x: x.get("content", "").rstrip().endswith("TERMINATE"),
)


## Perform a task

We invoke the `initiate_chat()` method of the user proxy agent to start the conversation. When you run the cell below, you will be prompted to provide feedback after receiving a message from the assistant agent. If you don't provide any feedback (by pressing Enter directly), the user proxy agent will try to execute the code suggested by the assistant agent on behalf of you, or terminate if the assistant agent sends a "TERMINATE" signal at the end of the message.

In [6]:
 math_problem_to_solve = """
Find $a + b + c$, given that $x+y \\neq -1$ and
\\begin{align}
	ax + by + c & = x + 7,\\
	a + bx + cy & = 2x + 6y,\\
	ay + b + cx & = 4x + y.
\\end{align}.
"""

# the assistant receives a message from the user, which contains the task description
user_proxy.initiate_chat(assistant, message=math_problem_to_solve)

user_proxy (to assistant):


Find $a + b + c$, given that $x+y \neq -1$ and 
\begin{align}
	ax + by + c & = x + 7,\
	a + bx + cy & = 2x + 6y,\
	ay + b + cx & = 4x + y.
\end{align}.


--------------------------------------------------------------------------------
assistant (to user_proxy):

 I see three linear equations given by the system: Ax + By + C = X + 7, Ax + By + C = 2X + 6Y, and Ay + B + C = 4X + Y. In order to find the values of a, b, and c, we need to solve this system of linear equations using Python. Here is the code:

```python
# filename: linear_equations.py
import numpy as np

coefficients = np.array([[1, 1, 1], [1, 1, -1], [a, b, c]])
constants = np.array([x + 7, 2 * x + 6 * y, 4 * x + y])
solution = np.linalg.solve(coefficients, constants)
a, b, c = solution

print("The values of a, b, and c are:")
print(f"a = {a}, b = {b}, c = {c}")
```

Replace x with the given value before executing this script. Make sure to have NumPy library installed in your Python environment f

user_proxy (to assistant):

exitcode: 1 (execution failed)
Code output: 
Traceback (most recent call last):
  File "linear_equations_sympy.py", line 16, in <module>
    sol = sp.solveset([eq1, eq2, eq3], [a, b, c])
  File "/usr/local/lib/python3.10/dist-packages/sympy/solvers/solveset.py", line 2214, in solveset
    raise ValueError("%s is not a valid SymPy expression" % f)
ValueError: [False, False, False] is not a valid SymPy expression


--------------------------------------------------------------------------------
assistant (to user_proxy):

 It seems there is an issue with the given system of linear equations, as the SymPy library cannot process a list of Boolean values. Let's check the given conditions and ensure they represent valid mathematical expressions before attempting to solve the system again.

Given conditions: x + y ≠ -1, ax + by + c = x + 7, a + bx + cy = 2x + 6y, ay + b + cx = 4x + y

First, let's ensure these conditions represent valid mathematical expressions by 

## Analyze the conversation

The human user can provide feedback at each step. When the human user didn't provide feedback, the code was executed. The executed results and error messages are returned to the assistant, and the assistant is able to modify the code based on the feedback. In the end, the task is complete and a "TERMINATE" signal is sent from the assistant. The user skipped feedback in the end and the conversation is finished.

After the conversation is finished, we can save the conversations between the two agents. The conversation can be accessed from `user_proxy.chat_messages`.

In [7]:
print(user_proxy.chat_messages[assistant])

[{'content': '\nFind $a + b + c$, given that $x+y \\neq -1$ and \n\\begin{align}\n\tax + by + c & = x + 7,\\\n\ta + bx + cy & = 2x + 6y,\\\n\tay + b + cx & = 4x + y.\n\\end{align}.\n', 'role': 'assistant'}, {'content': ' I see three linear equations given by the system: Ax + By + C = X + 7, Ax + By + C = 2X + 6Y, and Ay + B + C = 4X + Y. In order to find the values of a, b, and c, we need to solve this system of linear equations using Python. Here is the code:\n\n```python\n# filename: linear_equations.py\nimport numpy as np\n\ncoefficients = np.array([[1, 1, 1], [1, 1, -1], [a, b, c]])\nconstants = np.array([x + 7, 2 * x + 6 * y, 4 * x + y])\nsolution = np.linalg.solve(coefficients, constants)\na, b, c = solution\n\nprint("The values of a, b, and c are:")\nprint(f"a = {a}, b = {b}, c = {c}")\n```\n\nReplace x with the given value before executing this script. Make sure to have NumPy library installed in your Python environment for this code to run properly.\n\nThis code uses the numpy

In [8]:
import json

json.dump(user_proxy.chat_messages[assistant], open("conversations.json", "w"), indent=2)